#  What is TensorFlow ?
TensorFlow is a library for numerical computation where data flows through the graph.  Data in TensorFlow is represented by n-dimensional arrays called Tensors. Graph is made of data(Tensors) and mathematical operations. 

    Nodes on the graph: represent mathematical operations. 

    Edges on the graph: represent the Tensors that flow between operations.

In [7]:
import tensorflow as tf
print("tensorflow version: ",tf.__version__),

tensorflow version:  1.7.0


(None,)

# Graph in TensorFlow

Graph is the backbone of TensorFlow and every computation/operation/variables reside on the graph. Everything that happens in the code, resides on a default graph provided by TensorFlow

In [8]:
graph = tf.get_default_graph()
print(graph.get_operations())

[]


# TensorFlow session

A graph is used to define operations, but the operations are only run within a session. Graphs and sessions are created independently of each other. You can imagine graph to be similar to a blueprint, and a session to be similar to a construction site.

Graph only defines the computations or builds the blueprint. However, there are no variables, no values unless we run the graph or part of the graph within a session.

In [9]:
sess=tf.Session()
print("Run code what you want")
sess.close()


#another way to to do inside the Session
with tf.Session() as sess:
    print("Run code whatever you want")

Run code what you want
Run code whatever you want


# Tensor in TensorFlow

TF holds data in Tensors which are similar to numPy multi-dimensional arrays(although they are different from numPy Arrays):

# (a) Constants

are constants whose value can’t be changed. You can declare a constant like this: 

In [10]:
a=tf.constant(5.0)
a
print(a)

Tensor("Const:0", shape=(), dtype=float32)


# (b) Variable
are again Tensors which are like variables in any other language

In [11]:
b=tf.Variable(5.0,name="test_var")
b
#They need to be separately initialized by init op.
init_op=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run(b))

5.0


# Placeholders:
are tensors which are waiting to be initialized/fed. Placeholders are used for training data which is only fed when the code is actually run inside a session. What is fed to Placeholder is called feed_dict. Feed_dict are key value pairs for holding data:

In [13]:
c=tf.placeholder("float")
d=tf.placeholder("float")
y=tf.multiply(c,d)

 # Earlier this used to be tf.mul which has changed with Tensorflow 1.0
 # Typically we load feed_dict from somewhere else, 
 # may be reading from a training data folder etc
 # For simplicity, we have put values in feed_dict here

feed_dict={c:2,d:3}
with tf.Session() as sess:
    print(sess.run(y,feed_dict))

6.0


# Device in TensorFlow
TensorFlow has very strong in-built capabilites to run your code on a gpu or a cpu or a cluster of gpu etc. It provides you options to select the device you want to run your code


#  Simple example

### Reduce_mean

In [14]:
x=tf.Variable([10,20,30,40,50,60],name='t')
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(tf.reduce_mean(x)))

35


## Linear Regression Exercise

Problem statement: In linear regression, you get a lot of data-points and try to fit them on a straight line. For this example, we will create 100 datapoints and try to fit them into a line

### a) Creating training data
trainX has values between -1 and 1, and trainY has 3 times the trainX and some randomness.

In [16]:
import tensorflow as tf
import numpy as np
trainX=np.linspace(-1,1,101)
trainY=3*trainX + np.random.randn(*trainX.shape)*0.3
print(trainX)
print(trainY)

[-1.   -0.98 -0.96 -0.94 -0.92 -0.9  -0.88 -0.86 -0.84 -0.82 -0.8  -0.78
 -0.76 -0.74 -0.72 -0.7  -0.68 -0.66 -0.64 -0.62 -0.6  -0.58 -0.56 -0.54
 -0.52 -0.5  -0.48 -0.46 -0.44 -0.42 -0.4  -0.38 -0.36 -0.34 -0.32 -0.3
 -0.28 -0.26 -0.24 -0.22 -0.2  -0.18 -0.16 -0.14 -0.12 -0.1  -0.08 -0.06
 -0.04 -0.02  0.    0.02  0.04  0.06  0.08  0.1   0.12  0.14  0.16  0.18
  0.2   0.22  0.24  0.26  0.28  0.3   0.32  0.34  0.36  0.38  0.4   0.42
  0.44  0.46  0.48  0.5   0.52  0.54  0.56  0.58  0.6   0.62  0.64  0.66
  0.68  0.7   0.72  0.74  0.76  0.78  0.8   0.82  0.84  0.86  0.88  0.9
  0.92  0.94  0.96  0.98  1.  ]
[-2.84081054 -3.09440847 -3.51792313 -2.71674174 -2.97171159 -2.56883485
 -3.22529907 -2.97254135 -2.74839525 -2.6432536  -2.26678622 -2.52541113
 -2.07455843 -2.52782066 -2.67445874 -1.81846385 -2.09328451 -1.49231643
 -1.7851818  -1.16516425 -1.59657667 -2.13604057 -2.19589065 -1.06666045
 -1.94730611 -1.46654991 -1.14827031 -1.32786454 -1.67381496 -0.81008505
 -0.84003653 -1.48366

### b) Placeholders:

In [18]:
X=tf.placeholder("float")
Y=tf.placeholder("float")

### c) Modeling

Linear regression model :
y_model=w*x ,
Lets initialize w to 0 and 
cost=(Y-y_model)^2

We are going to define the training operation as changing the values using GradientDescentOptimizer to minimize cost with a learning rate of 0.01. Later we will run this training operation in a loop.


In [20]:
w=tf.Variable(0.0,name="weights")
y_model = tf.multiply(X,w)

cost= (tf.pow(Y-y_model,2))
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(cost)
print("train_op: ",train_op)


train_op:  name: "GradientDescent_1"
op: "NoOp"
input: "^GradientDescent_1/update_weights_2/ApplyGradientDescent"



### d) Training:
we have only defined the graph. No computation has happened.

None of the TensorFlow variables have any value. In order to run this graph, we need to create a Session and run. Before that we need to create the init_op to initialize all variables

In [21]:
# Creating init to initdialize all variables
init=tf.global_variables_initializer() 
# labeledData=zip(trainX,trainY)
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        for (x,y) in zip(trainX,trainY):
            sess.run(train_op,feed_dict={X:x,Y:y})
    print(sess.run(w))

    

3.006393


In [22]:
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(w))

0.0


# 1. What is a TensorFlow model
After you have trained a neural network, you would want to save it for future use and deploying to production. So, what is a Tensorflow model? Tensorflow model primarily contains the network design or graph and values of the network parameters that we have trained.
### a) Meta graph
This is a protocol buffer which saves the complete TensorFlow graph
### b) Checkpoin file:
this is a binary file which contains all the values of the weights, biases,gredients and all the other vaiables saved.

#### mymodel.data-00000-of-00001
#### mymodel.index

Here .data file is the file that contains our training variables.
Tensorflow also has a file named checkpoint which simply keeps a record of latest checkpoint files saved.

# 2. Saving a Tensorflow model
Once you see that the network has converged, you can stop the training manually or you will run the training for fixed number of epochs. After the training is done, we want to save all the variables and network graph to a file for future use.

In [16]:
import tensorflow as tf
w1 = tf.Variable(tf.random_normal(shape=[2]), name='w1')
w2 = tf.Variable(tf.random_normal(shape=[5]), name='w2')
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver.save(sess, 'my_test_model')

'my_test_model'

# Importing a pre-trained model:

In [18]:
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('my_test_model.meta')
    new_saver.restore(sess, tf.train.latest_checkpoint('./'))

INFO:tensorflow:Restoring parameters from ./my_test_model


In [20]:
 
with tf.Session() as sess:    
    saver = tf.train.import_meta_graph('my_test_model.meta')
    saver.restore(sess,tf.train.latest_checkpoint('./'))
    print(sess.run('w1:0'))
##Model has been restored. Above statement will print the saved value of w1.

INFO:tensorflow:Restoring parameters from ./my_test_model
[ 2.16286111  0.05795197]
